# Exploratory Data Analysis
---

**Objective:** Understand the variables, association between them and necessary transformations before model selection/building.

**Process:**
1. Importing libraries and dataset
2. Checking data types, nulls and outliers
3. Univariate analysis (target)
4. Bivariate analysis (relation to target)
5. Multivariate analysis (multicollinearity)
---

## 1. Importing libraries and dataset

In [7]:
# Adding src to the path for modules
import sys
import os
root_path = os.path.abspath('..')
src_path = os.path.join(root_path, 'src')
sys.path.append(src_path)

# Data Manipulation
import pandas as pd
import numpy as np

# Hypothesis Test
import scipy.stats as stats

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Project variables and functions
import config 
import utils
import importlib
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\vsuar\\OneDrive\\Documentos\\IncomePrediction\\src\\utils.py'>

Importing the data and having a first look at it:

In [2]:
# Importing
df = utils.read_compressed_data(config.COLLECTED_DATA_PATH)

# First look
df.sample(10)

,year,quarter,state,age,sex,race,literate,highest_educational_level,years_studied,worker_type,work_segment,occupation_group,tax_payer,weekly_worked_hours,main_work_income,all_work_income,weekly_worked_hours_all_jobs
23509,2022,4,CE,47,1,4,1,2,4,7,9,5,1,3,1212.0,1212.0,44
620859,2022,1,MG,59,1,4,1,1,0,9,1,6,2,1,3000.0,3000.0,14
932496,2023,2,MA,59,1,4,1,7,16,6,9,2,2,2,2300.0,2300.0,20
1423881,2021,1,PR,43,2,4,1,5,12,1,7,9,1,3,3000.0,3000.0,44
1832192,2021,4,RO,31,2,1,1,5,12,9,7,11,2,3,3500.0,3500.0,44
132720,2023,3,GO,33,1,4,2,2,2,1,4,5,1,3,1800.0,1800.0,40
1101160,2021,1,GO,66,1,1,1,2,6,9,10,7,2,3,1600.0,1600.0,40
1530161,2022,1,TO,70,1,4,2,1,0,9,4,5,2,4,1000.0,1000.0,48
437846,2023,1,PR,42,2,1,1,5,12,3,11,5,1,3,1500.0,1500.0,44
823530,2021,3,RJ,52,2,1,1,5,12,4,11,9,2,2,700.0,700.0,30


---
## 2. Checking data types, nulls and outliers

Checking types:

In [3]:
# Types
df.dtypes

year                              int64
quarter                           int64
state                            object
age                               int64
sex                               int64
race                              int64
literate                          int64
highest_educational_level         int64
years_studied                     int64
worker_type                       int64
work_segment                      int64
occupation_group                  int64
tax_payer                         int64
weekly_worked_hours               int64
main_work_income                float64
all_work_income                 float64
weekly_worked_hours_all_jobs      int64
dtype: object

Although almost all columns are codified as numeric, most of them are truly categorical.    
The ones that are numeric should be stored in a separate variable.

In [4]:
truly_numeric = ['year', 'quarter', 'age', 'years_studied', 'main_work_income', 'all_work_income']

Checking nulls:

In [5]:
# Nulls
utils.nulls_percentage(df)

Null Percentage by Column: 
------------------------------------------------------------
year                            0.0
quarter                         0.0
state                           0.0
age                             0.0
sex                             0.0
race                            0.0
literate                        0.0
highest_educational_level       0.0
years_studied                   0.0
worker_type                     0.0
work_segment                    0.0
occupation_group                0.0
tax_payer                       0.0
weekly_worked_hours             0.0
main_work_income                0.0
all_work_income                 0.0
weekly_worked_hours_all_jobs    0.0
dtype: float64


Checking outliers:

In [6]:
utils.outlier_percentage(df[truly_numeric])

Percentage of Outliers per Column: 


,Inner Fence,Outer Fence
year,0.00%,0.00%
quarter,0.00%,0.00%
age,0.07%,0.00%
years_studied,3.07%,0.00%
main_work_income,10.46%,5.56%
all_work_income,11.06%,5.94%


Inner Fence showed itself to be a lot more sensitive to outliers.   
Using outer fence keeps roughly 95% of income as it is right now, and probably will be the preferred method.   
I'll check the description to understand where these outliers may be.

In [8]:
utils.formatted_description(df[truly_numeric])

,year,quarter,age,years_studied,main_work_income,all_work_income
mean,2022.1,2.6,40.7,10.7,2417.7,2489.3
std,0.8,1.1,13.5,4.3,3660.5,3894.9
min,2021.0,1.0,14.0,0.0,2.0,2.0
25%,2021.0,2.0,30.0,8.0,1100.0,1100.0
50%,2022.0,3.0,40.0,12.0,1500.0,1500.0
75%,2023.0,4.0,51.0,13.0,2500.0,2500.0
max,2023.0,4.0,114.0,16.0,300000.0,900000.0


Doing a quick math, I see that inner fence was more sensitive to rows with few years studied and high income.    
I consider the intervals to be acceptable and therefore confirm my option for the outer fence method for dealing with outliers. 